# Trabajo Practico 2

## Carga de datos

In [152]:
import pandas as pd

# Creamos una variable con la ruta
ruta_archivo = "servicio_meteorológico_no.csv"

# Cargamos los datos en el dataframe con el enconding utf8
df = pd.read_csv(ruta_archivo, encoding="latin1")

# Mostramos las primeras filas y las ultimas para verificar
print(df.head())
print(df.tail())

         Estacin                 Valor Medio de   Ene   Feb   Mar   Abr  \
0  LA QUIACA OBS.         Temperatura (grados C)  12.8  12.5  12.4  10.9   
1  LA QUIACA OBS.  Temperatura mxima (grados C)  20.1  19.9  19.9  19.7   
2  LA QUIACA OBS.  Temperatura mnima (grados C)   7.4   7.0   6.3   2.8   
3  LA QUIACA OBS.           Humedad relativa (%)  65.3  64.1  62.4  48.2   
4  LA QUIACA OBS.    Velocidad del Viento (km/h)   6.9   7.1   6.5   6.4   

    May   Jun   Jul   Ago   Sep   Oct   Nov   Dic  
0   6.9   4.4   4.1   6.7   9.3  11.7  12.8  13.2  
1  17.3  15.6  15.4  17.4  19.2  21.1  21.7  21.3  
2  -3.0  -6.2  -6.8  -4.2  -1.1   2.8   5.2   6.9  
3  34.6  31.1  30.1  31.7  34.4  45.3  52.3  59.2  
4   6.0   4.9   6.6   6.6   8.6   8.7   8.8   8.3  
            Estacin                                     Valor Medio de   Ene  \
587  BASE SAN MARTIN                               Humedad relativa (%)   S/D   
588  BASE SAN MARTIN                        Velocidad del Viento (km/

## Limpieza de datos

In [153]:
# Creamos un diccionario con las palabras a reemplazar
reemplazos = {
    'mxima': 'máxima',
    'mnima': 'mínima',
    'Precipitacin': 'Precipitación',
    'das': 'días',
}
# Aca remplazamos las palabras con errores asignandole el diccionario creado
df.rename(columns={'Estacin': 'Estación'}, inplace=True)
# Aca este en particular no andaba con el replace.
df = df.replace(reemplazos, regex=True)
# Aca cambiamos los S/D que es sin datos a valor nulo
df.replace("S/D", np.nan, inplace=True)
# Primero podemos verificar el tema de los valores nulos
df = df.dropna() # PREGUNTAR HAY QUE BORRAR LOS NULOS?
# Tambien podemos verificar el tema de las filas duplicadas
df = df.drop_duplicates()
print(df.tail())


            Estación                 Valor Medio de   Ene   Feb   Mar   Abr  \
584  BASE SAN MARTIN         Temperatura (grados C)   2.0   0.9  -1.2  -3.3   
585  BASE SAN MARTIN  Temperatura máxima (grados C)   4.5   3.3   1.3  -0.9   
586  BASE SAN MARTIN  Temperatura mínima (grados C)  -0.3  -1.3  -3.6  -5.8   
588  BASE SAN MARTIN    Velocidad del Viento (km/h)  14.6  17.7  22.3  20.4   
589  BASE SAN MARTIN      Nubosidad total (octavos)   6.2   6.5   6.3   6.0   

      May    Jun    Jul    Ago    Sep    Oct   Nov   Dic  
584  -5.3   -9.3  -11.6  -11.5   -8.9   -5.9  -2.0   0.8  
585  -2.3   -5.8   -7.3   -6.9   -4.6   -2.1   1.2   3.4  
586  -8.4  -13.5  -16.3  -16.8  -13.9  -10.2  -5.5  -1.8  
588  19.3   18.0   17.3   18.9   17.5   20.3  19.2  18.3  
589   5.8    5.2    5.5    5.9    6.3    6.4   6.4   6.2  


## Analisis descriptivo

In [162]:
import statistics as st
import matplotlib.pyplot as plt

resumen_estadisticas = df.describe()
#print(resumen_estadisticas)

df_enero_diciembre = df[df.columns[2:]].transpose()
#print(df_enero_diciembre)

df_enero_diciembre.columns = df["Valor Medio de"]
#print(df_enero_diciembre)

df_enero_diciembre = df_enero_diciembre.loc["Ene":"Dic"]
#print(df_enero_diciembre)

temperatura_media = df_enero_diciembre[
    df_enero_diciembre.index == "Temperatura (grados C)"
].transpose()
temperatura_maxima = df_enero_diciembre[
    df_enero_diciembre.index == "Temperatura máxima (grados C)"
].transpose()
temperatura_minima = df_enero_diciembre[
    df_enero_diciembre.index == "Temperatura mínima (grados C)"
].transpose()
humedad_relativa = df_enero_diciembre[
    df_enero_diciembre.index == "Humedad relativa (%)"
].transpose()
velocidad_viento = df_enero_diciembre[
    df_enero_diciembre.index == "Velocidad del Viento (km/h)"
].transpose()

variables = [
    temperatura_media,
    temperatura_maxima,
    temperatura_minima,
    humedad_relativa,
    velocidad_viento,
]

for variable in variables:
    if not variable.empty:
        estacion = variable.columns[0]
        valores_numericos = pd.to_numeric(variable[estacion], errors="coerce").dropna()

        media = st.mean(valores_numericos)
        mediana = st.median(valores_numericos)
        desviacion_estandar = st.stdev(valores_numericos)

        print(f"Estadísticas para {estacion}:")
        print(f"  Media: {media}")
        print(f"  Mediana: {mediana}")
        print(f"  Desviación estándar: {desviacion_estandar}")
        print("\n")
    else:
        print(
            f"No hay datos para {variable.index[0]} en el rango de meses seleccionado."
        )

for variable in variables:
    if not variable.empty:  # Verificar si el DataFrame no está vacío
        estacion = variable.columns[0]
        valores_numericos = pd.to_numeric(variable[estacion], errors="coerce").dropna()

        plt.figure(figsize=(8, 6))
        plt.boxplot(valores_numericos)
        plt.title(f"Boxplot de {estacion}")
        plt.show()
    else:
        print(
            f"No hay datos para {variable.index[0]} en el rango de meses seleccionado."
        )

No hay datos para Temperatura (grados C) en el rango de meses seleccionado.
No hay datos para Temperatura (grados C) en el rango de meses seleccionado.
No hay datos para Temperatura (grados C) en el rango de meses seleccionado.
No hay datos para Temperatura (grados C) en el rango de meses seleccionado.
No hay datos para Temperatura (grados C) en el rango de meses seleccionado.
No hay datos para Temperatura (grados C) en el rango de meses seleccionado.
No hay datos para Temperatura (grados C) en el rango de meses seleccionado.
No hay datos para Temperatura (grados C) en el rango de meses seleccionado.
No hay datos para Temperatura (grados C) en el rango de meses seleccionado.
No hay datos para Temperatura (grados C) en el rango de meses seleccionado.
